In [ ]:
def main():
    dir = "LogoDet-3K/LogoDet-3K"
    test_split = 1/10
    val_split = 1/10

    train_data_list, val_data_list, test_data_list = getPathsSetsByBrand(dir, val_split, test_split, 1000, 15)

    print(f"images in the training set: {len(train_data_list)}")
    print(f"images in the validation set: {len(val_data_list)}")
    print(f"images in the test set: {len(test_data_list)}")

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    train_dataset = DatasetContrastive(train_data_list, transform)
    val_dataset = DatasetContrastive(val_data_list, transform)
    test_dataset = DatasetContrastive(test_data_list, transform)

    sample_idx = random.randint(0, len(train_dataset) - 1)
    img1, img2, is_positive = train_dataset[sample_idx]

    print(f"THE MATCH IS {is_positive}")





    return

if __name__ == "__main__":
    main()